In [1]:
# Copyright 2019 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

<img src="http://developer.download.nvidia.com/compute/machine-learning/frameworks/nvidia_logo.png" style="width: 90px; float: right;">

# DLRM Training and Inference Demo

## Overview


DLRM is a deep learning based approach to recommendation introduced by Facebook. 
Like other deep learning based approaches, DLRM is designed to make use of both categorical and numerical inputs which are usually present in RecSys training data. The architecture of DLRM can be understood via Figure 1. In order to handle categorical data, embedding layers map each category to a dense representation before being fed into  dense multilayer perceptrons (MLP). Continuous features can be fed directly into a dense MLP. At the next level, second-order interactions of different features are computed explicitly by taking the dot product between all pairs of embedding vectors and processed dense features. Those pairwise interactions are fed into a top level MLP to compute the likelihood of interaction between users and items. 

Compared to other DL based approaches to recommendation, DLRM differs in two ways. First, DLRM computes the feature interaction explicitly while limiting the order of interaction to pairwise interactions. Second, DLRM treats each embedded feature vector (corresponding to categorical features) as a single unit, whereas other methods treat each element  in the feature vector as a new unit that should yield different cross terms. These design choices help reduce computational/memory cost while maintaining competitive accuracy.

![DLRM_model](DLRM_architecture.png)

Figure 1. DLRM architecture.

### Learning objectives

This notebook demonstrates the steps for training a DLRM model. We then employ the trained model to make inference on new data.

## Content
1. [Requirements](#1)
1. [Data download and preprocessing](#2)
1. [Training](#3)
1. [Testing trained model](#4)


<a id="1"></a>
## 1. Requirements


### 1.1 Docker container
The most convenient way to make use of the NVIDIA DLRM model is via a docker container, which provides a self-contained, isolated and re-producible environment for all experiments. Refer to the [Quick Start Guide section](../README.md) of the Readme documentation for a comprehensive guide. We briefly summarize the steps here.

First, clone the repository:

```
git clone https://github.com/NVIDIA/DeepLearningExamples
cd DeepLearningExamples/PyTorch/Recommendation/DLRM
```

Next, build the DLRM container:
```
docker build . -t nvidia_dlrm_pyt
```

Make a directory for storing DLRM data and start a docker container with:
```
mkdir -p data
docker run --runtime=nvidia -it --rm --ipc=host  -v ${PWD}/data:/data nvidia_dlrm_pyt bash
```

Within the docker interactive bash session, start Jupyter with

```
export PYTHONPATH=/workspace/dlrm
jupyter notebook --ip 0.0.0.0 --port 8888
```

Then open the Jupyter GUI interface on your host machine at http://localhost:8888. Within the container, the demo notebooks are located at `/workspace/dlrm/notebooks`.

### 1.2 Hardware
This notebook can be executed on any CUDA-enabled NVIDIA GPU with at least 24GB of GPU memory, although for efficient mixed precision training, a [Tensor Core NVIDIA GPU](https://www.nvidia.com/en-us/data-center/tensorcore/) is desired (Volta, Turing or newer architectures). 

In [1]:
!nvidia-smi

Sat Mar 28 06:36:59 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   32C    P0    42W / 300W |      0MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   34C    P0    43W / 300W |      0MiB / 32510MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

<a id="2"></a>
## 2. Data download and preprocessing

Commercial recommendation systems are often trained on huge data sets, often in the order of terabytes, if not more. While datasets of this scale are rarely available to the public, the Criteo Terabyte click logs public [dataset](https://labs.criteo.com/2013/12/download-terabyte-click-logs/) offers a rare glimpse into the scale of real enterprise data: it contains ~1.3TB of uncompressed click logs collected over the course of 24 days, that can be used to train RecSys models that predict the ads click through rate. Yet, real datasets can be potentially one or two orders of magnitude larger, as enterprises will try to leverage as much historical data as they can use, for this will generally translate into better accuracy.

Herein, we employ the Criteo Terabyte dataset to demonstrate the efficiency of the GPU-optimized DLRM training procedure.  Each record in this dataset contains 40 columns: the first is a label column that indicates whether an user clicks an ad (value 1) or not (value 0). The next 13 columns are numeric, and the last 26 are categorical columns containing obfuscated hashed values. The columns and their values are all anonymized to protect user privacy.


We will first download and preprocess the Criteo Terabyte dataset. Note that this will require about 1TB of disk storage.

Notice: before downloading data, you must check out and agree with the terms and conditions of the Criteo Terabyte [dataset](https://labs.criteo.com/2013/12/download-terabyte-click-logs/).


In [ ]:
! cd ../preproc && ./prepare_dataset.sh

The original Facebook DLRM code base comes with a data preprocessing utility to preprocess the data. For continuous features, the data preprocessing steps include filling in missing values with 0 and normalization (shifting the values to be >=1 and taking natural logarithm). For categorical features, the preprocessing steps include building embedding tables and transforming hashed values into integer indicators. This code runs on a single CPU thread and takes ~6.5 days to transform the whole Criteo Terabyte data set. 

We improve the data preprocessing process with Spark on CPU to make use of all CPU threads. In the docker image, we have installed spark 2.4.5, which we’ll start a standalone Spark cluster.This results in significant improvement in data pre-processing speed, scaling approximately linearly with the number of available CPU threads. This outputs the transformed data in parquet format. We finally convert the parquet data into the binary format similar to that designed by the Facebook team specially for the Criteo dataset. 

Our preprocessing scripts are designed for the Criteo Terabyte Dataset and should work with any other dataset with the same format. The data should be split into text files. Each line of those text files should contain a single training example. An example should consist of multiple fields separated by tabulators:
- The first field is the label – `1` for a positive example and `0` for negative.
- The next `N` tokens should contain the numerical features separated by tabs.
- The next `M` tokens should contain the hashed categorical features separated by tabs.

The outcomes of the data preprocessing steps are by default stored in `/data/dlrm/binary_dataset` containing 3 binary data files: `test_data.bin`, `train_data.bin` and   `val_data.bin`  and a JSON `file model_size.json` totalling ~650GB.

Tips: by defaul the preprocessing script uses the first 23 days of the Criteo Terabyte dataset for training and the last day for validation. For a quick experiment, you can download and make use of a smaller number of days by modifying the `preproc/run_spark.sh` script.

<a id="3"></a>
## 3. Training

The repository provides several training recipes on 1 GPU with FP32 and automatic mixed precisions.

#### Training with FP32
Training on 1 GPU with FP32 with the `--nofp16` option.

In [ ]:
%run ../dlrm/scripts/main \
--mode train \
--dataset /data/dlrm/binary_dataset \
--nofp16 \
--save_checkpoint_path ./dlrm_model_fp32.pt

On a V100 32GB, training takes approximately 2h56m for 1 epoch to an AUC of ~0.8. The final result should look similar to the below.

```
Epoch:[0/1] [127600/128028]  eta: 0:00:34  loss: 0.1226  step_time: 0.080038  lr: 1.1766
Epoch:[0/1] [127800/128028]  eta: 0:00:18  loss: 0.1224  step_time: 0.080307  lr: 1.1480
Epoch:[0/1] [128000/128028]  eta: 0:00:02  loss: 0.1221  step_time: 0.080562  lr: 1.1199
Test: [200/2721]  loss: 0.1236  step_time: 0.0303
Test: [400/2721]  loss: 0.1248  step_time: 0.0245
Test: [600/2721]  loss: 0.1262  step_time: 0.0244
Test: [800/2721]  loss: 0.1262  step_time: 0.0245
Test: [1000/2721]  loss: 0.1293  step_time: 0.0245
Test: [1200/2721]  loss: 0.1307  step_time: 0.0245
Test: [1400/2721]  loss: 0.1281  step_time: 0.0245
Test: [1600/2721]  loss: 0.1242  step_time: 0.0246
Test: [1800/2721]  loss: 0.1230  step_time: 0.0245
Test: [2000/2721]  loss: 0.1226  step_time: 0.0244
Test: [2200/2721]  loss: 0.1239  step_time: 0.0246
Test: [2400/2721]  loss: 0.1256  step_time: 0.0249
Test: [2600/2721]  loss: 0.1247  step_time: 0.0248
Epoch 0 step 128027. Test loss 0.12557, auc 0.803517
Checkpoint saving took 42.90 [s]
DLL 2020-03-29 15:59:44.759627 - () best_auc : 0.80352  best_epoch : 1.00  average_train_throughput : 4.07e+05  average_test_throughput : 1.33e+06 
```

#### Training with mixed-precision
Mixed precision training can be done with the `--fp16` option. Under the hood, the NVIDIA Pytorch extension library [Apex](https://github.com/NVIDIA/apex) to enable mixed precision training.

Note: for subsequent launches of the %run magic, please restart your kernel manualy or execute the below cell to restart kernel.

In [ ]:
# Note: for subsequent launches of the %run magic, 
# please restart your kernel manualy or execute this cell to restart kernel.
import os
os._exit(00)

In [ ]:
%run ../dlrm/scripts/main \
--mode train \
--dataset /data/dlrm/binary_dataset \
--fp16 \
--save_checkpoint_path ./dlrm_model_fp16.pt

On a V100 32GB, training takes approximately 1h41m for 1 epoch to an AUC of ~0.8. Thus, mixed precision training provides a speed up of ~ 1.7x.

The final result should look similar to the below.

```
...
Epoch:[0/1] [127800/128028]  eta: 0:00:11  loss: 0.1224  step_time: 0.050719  lr: 1.1480
Epoch:[0/1] [128000/128028]  eta: 0:00:01  loss: 0.1221  step_time: 0.050499  lr: 1.1199
Test: [200/2721]  loss: 0.1236  step_time: 0.0271
Test: [400/2721]  loss: 0.1247  step_time: 0.0278
Test: [600/2721]  loss: 0.1262  step_time: 0.0275
Test: [800/2721]  loss: 0.1262  step_time: 0.0278
Test: [1000/2721]  loss: 0.1293  step_time: 0.0273
Test: [1200/2721]  loss: 0.1306  step_time: 0.0264
Test: [1400/2721]  loss: 0.1281  step_time: 0.0281
Test: [1600/2721]  loss: 0.1242  step_time: 0.0273
Test: [1800/2721]  loss: 0.1229  step_time: 0.0280
Test: [2000/2721]  loss: 0.1226  step_time: 0.0274
Test: [2200/2721]  loss: 0.1239  step_time: 0.0278
Test: [2400/2721]  loss: 0.1256  step_time: 0.0289
Test: [2600/2721]  loss: 0.1247  step_time: 0.0282
Epoch 0 step 128027. Test loss 0.12557, auc 0.803562
Checkpoint saving took 40.46 [s]
DLL 2020-03-28 15:15:36.290149 - () best_auc : 0.80356  best_epoch : 1.00  average_train_throughput : 6.47e+05  average_test_throughput : 1.17e+06
```

<a id="4"></a>
## 4. Testing trained model

After model training has completed, we can test the trained model against the Criteo test dataset. 

In [ ]:
# Note: for subsequent launches of the %run magic, 
# please restart your kernel manualy or execute this cell to restart kernel.
import os
os._exit(00)

In [ ]:
%run ../dlrm/scripts/main \
--mode test\
--dataset /data/dlrm/binary_dataset \
--load_checkpoint_path ./dlrm_model_fp16.pt

# Conclusion

In this notebook, we have walked through the complete process of preparing the container and data required for training the DLRM model. We have also investigated various training options with FP32 and automatic mixed precision, trained and tested DLRM models with new test data.

## What's next
Now it's time to try the DLRM model on your own data. Observe the performance impact of mixed precision training while comparing the final accuracy of the models trained with FP32 and mixed precision.
